<a href="https://colab.research.google.com/github/HuizhiXu/code-reading/blob/main/pseudo_crab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install xtal2png crabnet matbench ax-platform

In [ ]:
from xtal2png.utils.data import element_wise_scaler, element_wise_unscaler

$N$ as in number of attention layers (see [CrabNet hyperparameter manuscript](https://doi.org/10.1016/j.commatsci.2022.111505), Table 1).

In [ ]:
import pprint
def correct_parameterization(parameterization: dict, verbose=False):
    # take dictionary of tunable hyperparameters and output hyperparameter combinations compatible with CrabNet
    if verbose:
        pprint.pprint(parameterization)

    parameterization["out_hidden"] = [
        parameterization.get("out_hidden4") * 8,
        parameterization.get("out_hidden4") * 4,
        parameterization.get("out_hidden4") * 2,
        parameterization.get("out_hidden4"),
    ]
    parameterization.pop("out_hidden4")

    parameterization["betas"] = (
        parameterization.get("betas1"),
        parameterization.get("betas2"),
    )
    parameterization.pop("betas1")
    parameterization.pop("betas2")

    d_model = parameterization["d_model"]

    # make heads even (unless it's 1) (because d_model must be even)
    heads = parameterization["heads"]
    if np.mod(heads, 2) != 0:
        heads = heads + 1
    parameterization["heads"] = heads

    # NOTE: d_model must be divisible by heads
    d_model = parameterization["heads"] * round(d_model / parameterization["heads"])

    parameterization["d_model"] = d_model

    parameterization["pos_scaler_log"] = (
        1 - parameterization["emb_scaler"] - parameterization["pos_scaler"]
    )

    parameterization["epochs"] = parameterization["epochs_step"] * 4

    return parameterization

In [ ]:
import numpy as np

In [ ]:
def evaluate_surrogate(params):
  # import model



In [ ]:
from itertools import combinations
import numpy as np
import pandas as pd
from crabnet.crabnet_ import CrabNet
from matbench.bench import MatbenchBenchmark
from crabnet.utils.utils import count_parameters

# Helper fuction to slice the dictionary based on starting with string s
def slicedict(d, s):
    return {k:v for k,v in d.items() if k.startswith(s)}

# Helper fuction to validate the number of categorical variables and their values
def validate_categorical_inputs(c_dict):

    assert len(c_dict) <= 3, "Number of categorical variable should be less than or equal to 3"

    for k,v in c_dict.items():
        assert v == 0 or v == 1 or v == 2, "{} should be either 0, 1, or 2".format(k)


## Helper Function to convert input parameters to CrabNet hyperparameters
def userparam_To_crabnetparam(user_param, seed = 50):

    # set the seed
    np.random.seed(seed)


    # separating the integer/float from categorical variable
    x_dict = slicedict(user_param,'x')
    c_dict = slicedict(user_param,'c')

    # validate the number of categorical variables and their values
    validate_categorical_inputs(c_dict)

   #Defining Crabnet 3 categorical hyperparameters and their values
    crabnet_categorical = {'bias':['False','True'], 'criterion':['RobustL1','RobustL2'] , 'elem_prop':['mat2vec','magpie']}


    ## Defining Crabnet all 20 hyperparameters (integer/float) and their ranges
    crabnet_hyperparam = {
        'N':[1,10],'alpha':[0,1],'d_model':[100,1024], 'dim_feedforward':[1024,4096], 'dropout':[0,1],
        'emb_scaler':[0,1.0],'epochs_step':[5,20],'eps':[1e-7, 1e-4],'fudge':[0,1.0],'heads':[1,10],
        'k':[2,10],'lr':[1e-4, 6e-3],'pe_resolution':[2500, 10000],'ple_resolution':[2500, 10000],
        'pos_scaler':[0,1.0],'weight_decay':[0,1.0],'batch_size':[32,256],'out_hidden4':[32,512],
        'betas1':[0.5,0.9999],'betas2':[0.5,0.9999]

        }

    ### List of parameters having floating point values (total 10 nos)
    float_hyperparam = ['alpha','dropout','emb_scalar','eps','fudge','lr','pos_scaler','weight_decay','betas1','betas2']

    ## randomly selecting the parameters depending on x_dict size
    selected_param = np.random.choice(list(crabnet_hyperparam.keys()), size=len(x_dict), replace=False)

    ## randomly selecting the categorical parameters depending on c_dict size
    selected_catogorical = np.random.choice(list(crabnet_categorical.keys()), size=len(c_dict), replace=False)



    # Intializing actual param dict
    actual_param = {}


     # converting user param to Crabnet hyperparam (int/float)
    for p,k in zip(selected_param ,x_dict.keys()):

        # for handling hyperparameters having float values
        if p in float_hyperparam:
            actual_param[p] = np.float64(np.around(element_wise_scaler(x_dict[k], feature_range=crabnet_hyperparam[p], data_range=[0,1]),decimals=4,out=None))

        # for hyperparameters having integer values
        else:
            actual_param[p] = np.int(np.round(element_wise_scaler(x_dict[k], feature_range=crabnet_hyperparam[p], data_range=[0,1])))

    # converting categorical user param to Crabnet categorical hyperparam
    for s,c in zip(selected_catogorical,c_dict.values()):
        actual_param[s] = crabnet_categorical[s][c]


    return actual_param

def matbench_metric_calculator(crabnet_param):

    print('user parameters are :', crabnet_param)
    ## default hyperparameters
    parameterization = {
        'N':3,'alpha':0.5,'d_model':512, 'dim_feedforward':2048, 'dropout':0.1,
        'emb_scaler':1.0,'epochs_step':10,'eps':0.000001,'fudge':0.02,'heads':4,
        'k':6,'lr':0.001,'pe_resolution':5000,'ple_resolution':5000,
        'pos_scaler':1.0,'weight_decay':0,'batch_size':32,'out_hidden4':128,'betas1':0.9,'betas2':0.999,
        'losscurve' : False, 'learningcurve' : False, 'bias':False,'criterion':'RobustL1' , 'elem_prop':'mat2vec'

        }

    #*** replace rest of function with the evaluate_surrogate(params) function

    ## update the values of the selected hyperparameters
    parameterization.update(crabnet_param)

    print(parameterization)

    cb = CrabNet(**correct_parameterization(parameterization))

    mb = MatbenchBenchmark(autoload=False, subset = ["matbench_expt_gap"])

    for task in mb.tasks:
        task.load()
        for fold in task.folds:

            # Inputs are either chemical compositions as strings
            # or crystal structures as pymatgen.Structure objects.
            # Outputs are either floats (regression tasks) or bools (classification tasks)
            train_inputs, train_outputs = task.get_train_and_val_data(fold)

            # prep input for CrabNet
            train_df = pd.concat((train_inputs, train_outputs), axis=1, keys=["formula", "target"])

            if dummy:
                train_df = train_df.head(10)

            # train and validate your model
            cb.fit(train_df = train_df)

            # Get testing data
            test_inputs, test_outputs = task.get_test_data(fold, include_target=True)
            test_df = pd.concat((test_inputs, test_outputs), axis=1, keys=["formula", "target"])

            # Predict on the testing data
            # Your output should be a pandas series, numpy array, or python iterable
            # where the array elements are floats or bools
            predictions = cb.predict(test_df = test_df)

            predictions = np.nan_to_num(predictions)

            # Record your data!
            task.record(fold, predictions)

    model_size = count_parameters(cb.model)
    return({'average_mae':task.scores['mae']['mean'], 'average_rmse':task.scores['rmse']['mean'], "model_size": model_size}, crabnet_param)
    #return({'average_mae':task.scores['mae']['mean'], 'average_rmse':task.scores['rmse']['mean']}, crabnet_param)








In [ ]:
from numpy.testing import assert_array_less
import numpy as np

def check_float_ranges(parameters, tol=1e-6):
    xn_vals = np.array([[key, parameters.get(key)] for key in parameters.keys() if key.startswith("x")])
    num_vals = xn_vals.shape[0]

    ## checking float range (if val > 0 and val < 1 + 1e-6)   // ask if it should be val < -1.e6 instead
    passed = True

    assert_message = ""

    for key_val in xn_vals:

      key = key_val[0]
      val = key_val[1]

      if float(val) < 0 or float(val) > (1 + 1e-6):
        passed = False

        if len(assert_message) == 0:
          assert_message += f"{key}={val}"
        else:
          assert_message += f", {key}={val}"

    assert passed == True, "Parameters out of range. Failed due to the following parameter values: " + assert_message

check_float_ranges(dict(x1=0.5, x2=0.1, x3=0.2))

In [ ]:
dummy = True
user_param = {'x1':0.5,'x2':0.2,'x3':0.8,'x4':0.9,'c1':1,'c3':1,'c2':1}
user_param = {'x1':0.5,'x2':0.2,'x3':0.3,'c1':0,'c2':1}
actual_param = userparam_To_crabnetparam(user_param, seed = 50)
results = matbench_metric_calculator(actual_param)
#outputs = {"mae": results[0]["average_mae"], "rmse": results[0]["average_rmse"]}
#outputs


In [ ]:
actual_param

{'weight_decay': 0.5,
 'fudge': 0.2,
 'heads': 4,
 'criterion': 'RobustL1',
 'bias': 'True'}

In [ ]:
dummy = True #@param {type:"boolean"}

## PseudoCrab

Main public-facing class

In [ ]:
from typing import List, Optional
SUPPORTED_OBJECTIVES = ["mae", "rmse", "runtime", "model_size"]
####################
#SUPPORTED_OBJECTIVES = ["mae", "rmse"]
FLOAT_LIMIT = 20
CATEGORICAL_LIMIT = 3

def sum_constraint_fn(parameters, tol=1e-6):
  """Return True if x1 + x2 + ... + xn <= 1.0, else False."""

  xn_vals = np.array([parameters.get(key) for key in parameters.keys() if key.startswith("x")])

  print(xn_vals.sum())

  if xn_vals.sum() - tol <= 1.0:
    return True
  return False


def single_max_contraint_fn(parameters):
  """Return True if one of the parameters (x1, x2 ... xn) is >= 0.9, and x1 + x2 .. xn <= 1.0, else False."""

  xn_vals = np.array([parameters.get(key) for key in parameters.keys() if key.startswith("x")])

  if max(xn_vals) >= 0.9 and sum_constraint_fn(parameters):
      return True
  return False


class PseudoCrab(object):
  def __init__(self,
               objectives:List[str]=["mae"],
               iteration_budget: int=100,
               n_float_params: int=3,
               categorical_num_options: List[int] = [2, 2, 3],
               constraint_fn: Optional[callable]=sum_constraint_fn):

    for obj in objectives:
      assert obj in SUPPORTED_OBJECTIVES, f"Unsupported objective: {obj}. Must be in {SUPPORTED_OBJECTIVES}"

    self.objectives = objectives
    self.iteration_budget = iteration_budget

    # TODO: consider distinguishing between float and int parameters
    if n_float_params > FLOAT_LIMIT:
      raise ValueError(f"{n_float_params} float parameters requested. No more than {FLOAT_LIMIT} allowed.")
    self.n_float_params = n_float_params
    self.n_categorical_params = len(categorical_num_options)

    if self.n_categorical_params > CATEGORICAL_LIMIT:
      raise ValueError(f"{self.n_categorical_params} categorical parameters requested. No more than {CATEGORICAL_LIMIT} allowed.")

    self.constraint_fn = constraint_fn if constraint_fn is not None else lambda x: True

    self.n_objectives = len(objectives)
    self.expected_float_keys = [f"x{i}" for i in range(1, self.n_float_params + 1)]
    self.expected_categorical_keys = [f"c{i}" for i in range(1, self.n_categorical_params + 1)]
    self.expected_keys = self.expected_float_keys + self.expected_categorical_keys

    self.__num_evaluations = 0

  @property
  def num_evaluations(self):
    return self.__num_evaluations

  def evaluate(self, parameters):
    constraint_satisfied = self.constraint_fn(parameters)
    if not constraint_satisfied:
      # REVIEW: whether to raise a ValueError or return NaN outputs?
      raise ValueError(f"constraint_fn ({getattr(self.constraint_fn, '__name__', 'Unknown')}) not satisfied. Evaluation not counted towards budget.")

    check_float_ranges(parameters)

    self.__num_evaluations = self.num_evaluations + 1

    if self.num_evaluations > self.iteration_budget:
      raise ValueError("maximum number of evaluations has been reached")

    keys = list(parameters.keys())

    err_msg = ""
    missing_keys = np.setdiff1d(keys, self.expected_keys)
    if missing_keys:
      err_msg = err_msg + f"missing keys in parameters: {missing_keys}. "

    extra_keys = np.setdiff1d(self.expected_keys, keys)
    if extra_keys:
      err_msg = err_msg + f"extra keys in parameters: {extra_keys}. "

    if err_msg != "":
      raise KeyError(err_msg)

    crabnet_parameters = userparam_To_crabnetparam(parameters)

    results = matbench_metric_calculator(crabnet_parameters) #add try except block

    # # TODO: compute and return CrabNet objective(s) as dictionary
    # crabnet_mae = 0.123 # eV (dummy value)
    # crabnet_rmse = 0.234 # eV (dummy value)
    # runtime = 125 # seconds (dummy value)
    # model_size = 123456 # parameters (dummy value)

    outputs = {"mae": results[0]["average_mae"], "rmse": results[0]["average_rmse"]}

    # TODO: how to deal with runtime and memory? {"runtime": runtime, "model_size": model_size}

    return {k: outputs[k] for k in outputs.keys() if k in self.objectives}



class PseudoCrab1(PseudoCrab):
  def __init__(self):
   PseudoCrab.__init__(self, objectives = ["mae", "rmse"],
                       iteration_budget=100,
                       n_float_params=5,
                       categorical_num_options=[2,2,3],
                       constraint_fn=sum_constraint_fn
                      )

pc1 = PseudoCrab1()

## Example Usage (/Test)

The following shows intended usage of PseudoCrab to guide the development process. This is in the spirit of "write the tests for your code before you write your code".

In [ ]:
use_dummy_fn = False #@param {type:"boolean"}

if use_dummy_fn:
  objective_names = ["objective_1", "objective_2"]
  iteration_budget = 3
  n_float_params = 5
  n_categorical_params = 2
else:
  objective_names = ["mae", "rmse"]
  iteration_budget = 100 #@param {type:"integer"}
  n_float_params = 5 #@param {type: "integer"}
  n_categorical_params = 2 #@param {type: "integer"}

n_objectives = len(objective_names)


In [ ]:
# what about constraint function?
default_benchmarks = dict(
    dummy=dict(objective_names=["mae", "rmse"], iteration_budget=3, n_float_params=5, n_categorical_params=2),
    minimal=dict(objective_names=["mae", "rmse"], iteration_budget=100, n_float_params=5, n_categorical_params=0), # single-vs-multi objective?
    benchmark2=dict(objective_names=["mae", "rmse"], iteration_budget=100, n_float_params=5, n_categorical_params=2),
    # benchmark3=dict(objective_names=["mae", "rmse"], iteration_budget=100, n_float_params=5, n_categorical_params=2),
    # benchmark4=dict(objective_names=["mae", "rmse"], iteration_budget=100, n_float_params=5, n_categorical_params=2),
    # base alloy optimization benchmark and constraint? base alloy at least X%, remainder goes to other parameters
    performance=dict(objective_names=["mae", "rmse", "model_size"], constraint_fn=sum_constraint_fn, iteration_budget=100, n_float_params=23, n_categorical_params=3),
    )


class PseudoCrabMinimal(PseudoCrab):
  def __init__(self):
    PseudoCrab.__init__(
        self,
        objectives = ["mae"],
        iteration_budget = 100,
        n_float_params = 3,
        categorical_num_options = [],
        constraint_fn = sum_constraint_fn
    )

class PseudoCrabPerformance(PseudoCrab):
  def __init__(self):
    PseudoCrab.__init__(
        self,
        objectives = ["mae", "rmse"],
        iteration_budget = 100,
        n_float_params = 23,
        categorical_num_options = [2,2,3],
        constraint_fn = sum_constraint_fn
    )

In [ ]:
# TODO: implement some ConstraintFunction classes that let you specify
# max number of components that can be changed at a time, factory constraints for
# linear inequality, order, and bound constraints, etc.
def example_constraint_fn(parameters):
  """Return True if x1 + x2 <= 1.0, else False."""
  if parameters["x1"] + parameters["x2"] <= 1.0:
    return True
  return False

In [ ]:
# Testing the sum_contraint_fn function with different parameters cases
import numpy as np
parameters = {"x1":0.2, "x2":0.4, "x3":0.1, "x4":0.1, "x5":0.1, "x6":0.2, "c1":2, "c2":3}
sum_constraint_fn(parameters)

1.1


False

In [ ]:
# implement an Ax search here for an example, and create a dummy function to run in place of PseudoCrab for demo purposes
"""Adapted from https://ax.dev/docs/api.html and https://ax.dev/tutorials/multiobjective_optimization.html"""
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from botorch.test_functions.multi_objective import BraninCurrin  # for dummy fn
import torch  # for dummy fn
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models
import numpy as np
from copy import deepcopy

# Load our sample 2-objective problem
branin_currin = BraninCurrin(negate=True).to(
    dtype=torch.double,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)


def dummy_evaluate(parameters):
    evaluation = branin_currin(
        torch.tensor([parameters.get("x1"), parameters.get("x2")])
    )
    parameters["x3"]  # unused float parameter
    parameters["x4"]  # unused float parameter
    parameters["x5"]  # unused float parameter
    parameters["c1"]  # unused categorical parameter
    parameters["c2"]  # unused categorical parameter
    branin_measured = evaluation[0].item()
    currin_measured = evaluation[1].item()
    return {"objective_1": branin_measured, "objective_2": currin_measured}


if use_dummy_fn:
    if n_objectives != 2 or n_float_params != 5 or n_categorical_params != 2:
        raise ValueError(
            "dummy_evaluate is hardcoded for 2 objectives, 5 float parameters, and 2 categorical parameters"
        )
    evaluation_function = dummy_evaluate
else:
    pc = PseudoCrab(
        objectives=objective_names,
        # constraint_fn=example_constraint_fn,
        iteration_budget=iteration_budget,
        n_float_params=n_float_params,
        # n_categorical_params=n_categorical_params
        # complexity=2,
    )
    # REVIEW: encode some way of changing the complexity of the task (e.g. runtime cost, response surface smoothness)
    # e.g. for iteration_cost, specify how many training datapoints are used internally

    dummy_parameters = {
        **{f"x{i}": 0.1 for i in range(1, 6)},
        **{f"c{i}": 1 for i in range(1, 4)},
    }
    deepcopy(pc).evaluate(dummy_parameters)  # test that it runs with a simple case

    def evaluation_function(parameters):
      parameters["x5"] = 1 - sum([parameters[f"x{i}"] for i in range(1, 5)])
      return pc.evaluate(parameters)

composition_bounds = [0.0, 1.0]


def make_options(num_options):
    """E.g. make_options(3) == [0, 1, 2]"""
    return [i for i in range(num_options)]


def make_objectives(num_objectives):
    """E.g. make_objectives(2) == ["mae": ObjectiveProperties(minimize=True), "rmse": ObjectiveProperties(minimize=True)}"""
    return {name: ObjectiveProperties(minimize=True) for name in objective_names[:num_objectives]}


objectives = make_objectives(n_objectives)

if n_objectives == 1:
    model = Models.GPEI
else:
    model = Models.MOO
gs = GenerationStrategy(
    steps=[
        # 1. Initialization step (does not require pre-existing data and is well-suited for
        # initial sampling of the search space)
        GenerationStep(
            model=Models.SOBOL,
            num_trials=max(
                2, 5 - iteration_budget
            ),  # How many trials should be produced from this generation step
        ),
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=model,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
        ),
    ]
)

ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="pseudo_crab_experiment",
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "c1",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },
        {
            "name": "c2",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },

        {
            "name": "c3",
            "type": "choice",
            "values": make_options(
                3
            ),  # number of options should match corresponding CrabNet hyperparameter
        },
    ],
    objectives=objectives,

    parameter_constraints=["x1 + x2 + x3 + x4  <= 1.0"]
)

for _ in range(iteration_budget):
    parameters, trial_index = ax_client.get_next_trial()
    print('parameters   : ', parameters)
    ax_client.complete_trial(
        trial_index=trial_index, raw_data = evaluation_function(parameters)
    )

if n_objectives == 1:
    print("1 OBJECTIVE")
    best_parameters, metrics = ax_client.get_best_parameters()
    pareto_optimal_parameters = None
else:
    print("MULTIPLE OBJECTIVES")
    best_parameters = None
    pareto_optimal_parameters = ax_client.get_pareto_optimal_parameters()


In [ ]:
print(best_parameters)
print(pareto_optimal_parameters)

In [ ]:
# implement an Ax search here for an example, and create a dummy function to run in place of PseudoCrab for demo purposes
"""Adapted from https://ax.dev/docs/api.html and https://ax.dev/tutorials/multiobjective_optimizaotion.html"""

# This example is for finding optimal for alloy design having x1>=70%
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from botorch.test_functions.multi_objective import BraninCurrin  # for dummy fn
import torch  # for dummy fn
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models
import numpy as np
from copy import deepcopy

# Load our sample 2-objective problem
branin_currin = BraninCurrin(negate=True).to(
    dtype=torch.double,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)


def dummy_evaluate(parameters):
    evaluation = branin_currin(
        torch.tensor([parameters.get("x1"), parameters.get("x2")])
    )
    parameters["x3"]  # unused float parameter
    parameters["x4"]  # unused float parameter
    parameters["x5"]  # unused float parameter
    parameters["c1"]  # unused categorical parameter
    parameters["c2"]  # unused categorical parameter
    branin_measured = evaluation[0].item()
    currin_measured = evaluation[1].item()
    return {"objective_1": branin_measured, "objective_2": currin_measured}


if use_dummy_fn:
    if n_objectives != 2 or n_float_params != 5 or n_categorical_params != 2:
        raise ValueError(
            "dummy_evaluate is hardcoded for 2 objectives, 5 float parameters, and 2 categorical parameters"
        )
    evaluation_function = dummy_evaluate
else:
    pc = PseudoCrab(
        objectives=objective_names,
        constraint_fn= single_max_contraint_fn,
        iteration_budget=iteration_budget,
        n_float_params=n_float_params,
        # n_categorical_params=n_categorical_params
        # complexity=2,
    )
    # REVIEW: encode some way of changing the complexity of the task (e.g. runtime cost, response surface smoothness)
    # e.g. for iteration_cost, specify how many training datapoints are used internally

    dummy_parameters = {
        # **{f"x{i}": 0.1 for i in range(1, 6)},
        # **{f"c{i}": 1 for i in range(1, 3)},
        'x1':0.9,
        'x2':0.05,
        'x3':0.05,
        'x4':0,
        'x5':0,
        'c1':0,
        'c2':1,
        'c3':0

    }
    deepcopy(pc).evaluate(dummy_parameters)  # test that it runs with a simple case

    def evaluation_function(parameters):
        parameters["x5"] = 1 - sum([parameters[f"x{i}"] for i in np.arange(1, 5)])
        print('parameters:  ::::',parameters)
        return pc.evaluate(parameters)

composition_bounds = [0.0, 0.1]


def make_options(num_options):
    """E.g. make_options(3) == [0, 1, 2]"""
    return [i for i in range(num_options)]


def make_objectives(num_objectives):
    """E.g. make_objectives(2) == ["mae": ObjectiveProperties(minimize=True), "rmse": ObjectiveProperties(minimize=True)}"""
    return {name: ObjectiveProperties(minimize=True) for name in objective_names[:num_objectives]}


objectives = make_objectives(n_objectives)

if n_objectives == 1:
    model = Models.GPEI
else:
    model = Models.MOO
gs = GenerationStrategy(
    steps=[
        # 1. Initialization step (does not require pre-existing data and is well-suited for
        # initial sampling of the search space)
        GenerationStep(
            model=Models.SOBOL,
            num_trials=max(
                2, 5 - iteration_budget
            ),  # How many trials should be produced from this generation step
        ),
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=model,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
        ),
    ]
)

ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="pseudo_crab_experiment",
    parameters=[

        {
            "name": "x1",
            "type": "range",
            "bounds": [0.9,1.0]
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": composition_bounds,
        },

        {
            "name": "x4",
            "type": "range",
            "bounds": composition_bounds,
        },
        # {
        #     "name": "x5",
        #     "type": "range",
        #     "bounds": composition_bounds,
        # },
        {
            "name": "c1",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },
        {
            "name": "c2",
            "type": "choice",
            "values": make_options(
                2
            ),
        },
        {
            "name": "c3",
            "type": "choice",
            "values": make_options(
                2
            ),
        }

    ],
    objectives=objectives,

    parameter_constraints=["x1 + x2 + x3 + x4  <= 1"]
)

for _ in range(iteration_budget):
    parameters, trial_index = ax_client.get_next_trial()
    print('parameters :',parameters)
    ax_client.complete_trial(
        trial_index=trial_index, raw_data = evaluation_function(parameters)
    )

if n_objectives == 1:
    print("1 OBJECTIVE")
    best_parameters, metrics = ax_client.get_best_parameters()
    pareto_optimal_parameters = None
else:
    print("MULTIPLE OBJECTIVES")
    best_parameters = None
    pareto_optimal_parameters = ax_client.get_pareto_optimal_parameters()


With only SOBOL Search

In [ ]:
def matbench_metric_calculator(crabnet_param):

    print('user parameters are :', crabnet_param)
    ## default hyperparameters
    parameterization = {
        'N':3,'alpha':0.5,'d_model':512, 'dim_feedforward':2048, 'dropout':0.1,
        'emb_scaler':1.0,'epochs_step':10,'eps':0.000001,'fudge':0.02,'heads':4,
        'k':6,'lr':0.001,'pe_resolution':5000,'ple_resolution':5000,
        'pos_scaler':1.0,'weight_decay':0,'batch_size':32,'out_hidden4':128,'betas1':0.9,'betas2':0.999,
        'losscurve' : False, 'learningcurve' : False, 'bias':False,'criterion':'RobustL1' , 'elem_prop':'mat2vec'

        }

    ## update the values of the selected hyperparameters
    parameterization.update(crabnet_param)

    print(parameterization)

    cb = CrabNet(**correct_parameterization(parameterization))

    mb = MatbenchBenchmark(autoload=False, subset = ["matbench_expt_gap"])

    try:
        for task in mb.tasks:
            task.load()
            for fold in task.folds:

                # Inputs are either chemical compositions as strings
                # or crystal structures as pymatgen.Structure objects.
                # Outputs are either floats (regression tasks) or bools (classification tasks)
                train_inputs, train_outputs = task.get_train_and_val_data(fold)

                # prep input for CrabNet
                train_df = pd.concat((train_inputs, train_outputs), axis=1, keys=["formula", "target"])

                if dummy:
                    train_df = train_df.head(10)

                # train and validate your model
                cb.fit(train_df = train_df)

                # Get testing data
                test_inputs, test_outputs = task.get_test_data(fold, include_target=True)
                test_df = pd.concat((test_inputs, test_outputs), axis=1, keys=["formula", "target"])

                # Predict on the testing data
                # Your output should be a pandas series, numpy array, or python iterable
                # where the array elements are floats or bools
                predictions = cb.predict(test_df = test_df)

                predictions = np.nan_to_num(predictions)

                # Record your data!
                task.record(fold, predictions)

        model_size = count_parameters(cb.model)
        return({'average_mae':task.scores['mae']['mean'], 'average_rmse':task.scores['rmse']['mean'], "model_size": model_size}, crabnet_param)
        #return({'average_mae':task.scores['mae']['mean'], 'average_rmse':task.scores['rmse']['mean']}, crabnet_param)

    except IndexError as e:
        print("TODO:: Error encountered...")
        print(e)
        return({'average_mae':137890, 'average_rmse':1234, "model_size": 1678}, crabnet_param)


In [ ]:
# implement an Ax search here for an example, and create a dummy function to run in place of PseudoCrab for demo purposes
"""Adapted from https://ax.dev/docs/api.html and https://ax.dev/tutorials/multiobjective_optimization.html"""
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from botorch.test_functions.multi_objective import BraninCurrin  # for dummy fn
import torch  # for dummy fn
from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models
import numpy as np
from copy import deepcopy

# Load our sample 2-objective problem
branin_currin = BraninCurrin(negate=True).to(
    dtype=torch.double,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
)


def dummy_evaluate(parameters):
    evaluation = branin_currin(
        torch.tensor([parameters.get("x1"), parameters.get("x2")])
    )
    parameters["x3"]  # unused float parameter
    parameters["x4"]  # unused float parameter
    parameters["x5"]  # unused float parameter
    parameters["c1"]  # unused categorical parameter
    parameters["c2"]  # unused categorical parameter
    branin_measured = evaluation[0].item()
    currin_measured = evaluation[1].item()
    return {"objective_1": branin_measured, "objective_2": currin_measured}


if use_dummy_fn:
    if n_objectives != 2 or n_float_params != 5 or n_categorical_params != 2:
        raise ValueError(
            "dummy_evaluate is hardcoded for 2 objectives, 5 float parameters, and 2 categorical parameters"
        )
    evaluation_function = dummy_evaluate
else:
    pc = PseudoCrab(
        objectives=objective_names,
        # constraint_fn=example_constraint_fn,
        iteration_budget=100,
        n_float_params=n_float_params,
        # n_categorical_params=n_categorical_params
        # complexity=2,
    )
    # REVIEW: encode some way of changing the complexity of the task (e.g. runtime cost, response surface smoothness)
    # e.g. for iteration_cost, specify how many training datapoints are used internally

    dummy_parameters = {
        **{f"x{i}": 0.1 for i in range(1, 6)},
        **{f"c{i}": 1 for i in range(1, 4)},
    }
    deepcopy(pc).evaluate(dummy_parameters)  # test that it runs with a simple case

    def evaluation_function(parameters):
      parameters["x5"] = 1 - sum([parameters[f"x{i}"] for i in range(1, 5)])
      return pc.evaluate(parameters)

composition_bounds = [0.0, 1.0]


def make_options(num_options):
    """E.g. make_options(3) == [0, 1, 2]"""
    return [i for i in range(num_options)]


def make_objectives(num_objectives):
    """E.g. make_objectives(2) == ["mae": ObjectiveProperties(minimize=True), "rmse": ObjectiveProperties(minimize=True)}"""
    return {name: ObjectiveProperties(minimize=True) for name in objective_names[:num_objectives]}


objectives = make_objectives(n_objectives)

if n_objectives == 1:
    model = Models.GPEI
else:
    model = Models.MOO
gs = GenerationStrategy(
    steps=[
        # 1. Initialization step (does not require pre-existing data and is well-suited for
        # initial sampling of the search space)
        GenerationStep(
            model=Models.SOBOL,
            num_trials= -1)
    ]
)

ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="pseudo_crab_experiment",
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": composition_bounds,
        },
        {
            "name": "c1",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },
        {
            "name": "c2",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },

        {
            "name": "c3",
            "type": "choice",
            "values": make_options(
                2
            ),  # number of options should match corresponding CrabNet hyperparameter
        },
    ],
    objectives=objectives,

    parameter_constraints=["x1 + x2 + x3 + x4  <= 1.0"]
)

for _ in range(iteration_budget):
    parameters, trial_index = ax_client.get_next_trial()
    print('parameters   : ', parameters)
    ax_client.complete_trial(
        trial_index=trial_index, raw_data = evaluation_function(parameters)
    )

if n_objectives == 1:
    print("1 OBJECTIVE")
    best_parameters, metrics = ax_client.get_best_parameters()
    pareto_optimal_parameters = None
else:
    print("MULTIPLE OBJECTIVES")
    best_parameters = None
    pareto_optimal_parameters = ax_client.get_pareto_optimal_parameters()


0.5
user parameters are : {'weight_decay': 0.1, 'fudge': 0.1, 'heads': 2, 'betas1': 0.55, 'dim_feedforward': 1331, 'criterion': 'RobustL2', 'bias': 'True', 'elem_prop': 'magpie'}
{'N': 3, 'alpha': 0.5, 'd_model': 512, 'dim_feedforward': 1331, 'dropout': 0.1, 'emb_scaler': 1.0, 'epochs_step': 10, 'eps': 1e-06, 'fudge': 0.1, 'heads': 2, 'k': 6, 'lr': 0.001, 'pe_resolution': 5000, 'ple_resolution': 5000, 'pos_scaler': 1.0, 'weight_decay': 0.1, 'batch_size': 32, 'out_hidden4': 128, 'betas1': 0.55, 'betas2': 0.999, 'losscurve': False, 'learningcurve': False, 'bias': 'True', 'criterion': 'RobustL2', 'elem_prop': 'magpie'}

Model architecture: out_dims, d_model, N, heads
3, 512, 3, 2
Running on compute device: cpu
2022-11-28 22:42:03 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: http

2022-11-28 22:42:03 INFO     Loading dataset 'matbench_expt_gap'...


INFO:matbench.task:Loading dataset 'matbench_expt_gap'...


2022-11-28 22:42:03 INFO     Dataset 'matbench_expt_gap loaded.


...

Generating EDM: 100%|██████████| 920/920 [00:00<00:00, 68592.86formulae/s]


loading data with up to 4 elements in the formula
2022-11-29 00:26:58 INFO     Recorded fold matbench_expt_gap-4 successfully.


INFO:matbench.task:Recorded fold matbench_expt_gap-4 successfully.
[INFO 11-29 00:26:58] ax.service.ax_client: Completed trial 99 with data: {'mae': (0.994771, None), 'rmse': (1.443552, None)}.


MULTIPLE OBJECTIVES


[INFO 11-29 00:27:01] ax.service.utils.best_point: Using inferred objective thresholds: [ObjectiveThreshold(mae <= -16.628530238137046), ObjectiveThreshold(rmse <= 1.435833177014068)], as objective thresholds were not specified as part of the optimization configuration on the experiment.


## Notes

Consider making it so the ConstraintFunction classes can be added together. One caveat is the need to allow randomness in the assignment of variables and doing repeat runs to get better bounds on the transferability to the domain problem.

In the end, we probably don't know the underlying correlations between variables in the domain problem, so while we can make it "look" like the same optimization problem, we might not be able to make it "quack" very much like the same optimization problem. Domain knowledge can come into play in making the choices, but that introduces some new complexities. For example, assigning certain CrabNet hyperparameters to the corresponding PseudoCrab parameters based on how correlated they are with other hyperparameters.

## Code Graveyard

In [ ]:
        # test_df = test_inputs.to_frame(name="formula")
        # test_df["target"] = 0.0

# class RandomSearch(num_iterations=100, param_types: dict(x0="float", x1="float", x2="float", x3="float", x4="float", c0="string", c1="string")):
#   def __init__():
#     pass
#   def get_next_trial():


# from ax import optimize
# best_parameters, values, experiment, model = optimize(
#     parameters=[
#         {
#             "name": "x0",
#             "type": "range",
#             "bounds": [-5.0, 10.0],
#         },
#         {
#             "name": "x1",
#             "type": "range",
#             "bounds": [0.0, 10.0],
#         },
#         {
#             "name": "x2",
#             "type": "range",
#             "bounds": [-5.0, 10.0],
#         },
#         {
#             "name": "x3",
#             "type": "range",
#             "bounds": [0.0, 10.0],
#         },
#         {
#             "name": "x4",
#             "type": "range",
#             "bounds": [0.0, 10.0],
#         },
#     ],
#     evaluation_function=branin_evaluation_function,
#     minimize=True,
# )


# from ax.utils.measurement.synthetic_functions import branin
# def augmented_branin_evaluation_function(parameters):
#     mean = branin(parameters["x1"], parameters["x2"])
#     parameters["x3"]  # unused float parameter
#     parameters["x4"]  # unused float parameter
#     parameters["x5"]  # unused float parameter
#     parameters["c1"]  # unused categorical parameter
#     parameters["c2"]  # unused categorical parameter
#     # return std_dev is optional
#     # std = 0.0
#     # return (mean, std)
#     return {"objective_1": mean, "objective_2": mean}

# extra_objs = np.setdiff1d(objectives, SUPPORTED_OBJECTIVES)

In [ ]:
# N_pseudo = [0.1, 0.125, 0.01, 0.0, 0.95]
# N_pseudo_range = [0, 1.0]
# N_range = [1, 10]
# N_actual = element_wise_scaler(N_pseudo, feature_range=N_range, data_range=N_pseudo_range)
# N_actual = np.int64(np.round(N_actual))
# print('N_actual:',N_actual)

# d_model_pseudo = np.arange(0, 1, 0.2)
# d_model_pseudo_range = [0, 1.0]
# d_model_range = [100, 1024]
# d_model_actual = element_wise_scaler(d_model_pseudo, feature_range=d_model_range, data_range=d_model_pseudo_range)
# d_model_actual = np.int64(np.round(d_model_actual))
# print('d_model_actual:',d_model_actual)

# dim_feedforward_pseudo = np.arange(0, 1, 0.2)
# dim_feedforward_pseudo_range = [0, 1.0]
# dim_feedforward_range = [1024, 4096]
# dim_feedforward_actual = element_wise_scaler(dim_feedforward_pseudo, feature_range=dim_feedforward_range, data_range=dim_feedforward_pseudo_range)
# dim_feedforward_actual = np.int64(np.round(dim_feedforward_actual))
# print('dim_feedforward_actual:',dim_feedforward_actual)

# eps_pseudo = np.arange(0, 1, 0.2)
# eps_pseudo_range = [0, 1.0]
# eps_range = [1e-7, 1e-4]
# eps_actual = element_wise_scaler(eps_pseudo,feature_range=eps_range, data_range=eps_pseudo_range)
# eps_actual = np.float64(np.around(eps_actual, decimals=7, out=None))
# print('eps_actual:',eps_actual)

# fudge_pseudo = np.arange(0, 1, 0.2)
# fudge_pseudo_range = [0, 1.0]
# fudge_range = [0, 0.1]
# fudge_actual = element_wise_scaler(fudge_pseudo, feature_range=fudge_range, data_range=fudge_pseudo_range)
# print('fudge_actual:',fudge_actual)

# heads_pseudo = np.arange(0, 1, 0.2)
# heads_pseudo_range = [0, 1.0]
# heads_range = [1, 10]
# heads_actual = element_wise_scaler(heads_pseudo, feature_range=heads_range, data_range=heads_pseudo_range)
# heads_actual = np.int64(np.round(heads_actual))
# print('heads:',heads_actual)

# k_pseudo = np.arange(0, 1, 0.2)
# k_pseudo_range = [0, 1.0]
# k_range = [2, 10]
# k_actual = element_wise_scaler(k_pseudo, feature_range=k_range, data_range=k_pseudo_range)
# k_actual = np.int64(np.round(k_actual))
# print('k_actual:',k_actual)

# lr_pseudo = np.arange(0, 1, 0.2)
# lr_pseudo_range = [0, 1.0]
# lr_range = [1e-4, 6e-3]
# lr_actual = element_wise_scaler(lr_pseudo, feature_range=lr_range, data_range=lr_pseudo_range)
# lr_actual = np.float64(np.around(lr_actual, decimals=4, out=None)) # to check the floating point round off
# print('lr_actual:',lr_actual)

# pe_resolution_pseudo = np.arange(0, 1, 0.2)
# pe_resolution_pseudo_range = [0, 1.0]
# pe_resolution_range = [2500, 10000]
# pe_resolution_actual = element_wise_scaler(pe_resolution_pseudo, feature_range=pe_resolution_range, data_range=pe_resolution_pseudo_range)
# pe_resolution_actual = np.int64(np.round(pe_resolution_actual))
# print('pe_resolution_actual:',pe_resolution_actual)

# ple_resolution_pseudo = np.arange(0, 1, 0.2)
# ple_resolution_pseudo_range = [0, 1.0]
# ple_resolution_range = [2500, 10000]
# ple_resolution_actual = element_wise_scaler(ple_resolution_pseudo, feature_range=ple_resolution_range, data_range=ple_resolution_pseudo_range)
# ple_resolution_actual = np.int64(np.round(ple_resolution_actual))
# print('ple_resolution_actual:',ple_resolution_actual)

# batch_size_pseudo = np.arange(0, 1, 0.2)
# batch_size_pseudo_range = [0, 1.0]
# batch_size_range = [32, 256] ### (continuous or discrete like 32,64,128,256)
# batch_size_actual = element_wise_scaler(batch_size_pseudo, feature_range=batch_size_range,data_range=batch_size_pseudo_range)
# batch_size_actual = np.int64(np.round(batch_size_actual))
# print('batch_size_actual:',batch_size_actual)

# out_hidden4_pseudo = np.arange(0, 1, 0.2)
# out_hidden4_pseudo_range = [0, 1.0]
# out_hidden4_range = [32, 512]
# out_hidden_actual = element_wise_scaler(out_hidden4_pseudo, feature_range=out_hidden4_range, data_range=out_hidden4_pseudo_range)
# out_hidden_actual = np.int64(np.round(out_hidden_actual))
# print('out_hidden_actual:',out_hidden_actual)

# betas1_pseudo = np.arange(0, 1, 0.2)
# betas1_pseudo_range = [0, 1.0]
# betas1_range = [0.5, 0.9999]
# betas1_actual = element_wise_scaler(betas1_pseudo,feature_range=betas1_range,data_range=betas1_pseudo_range)
# betas1_actual = np.float64(np.around(betas1_actual, decimals=4, out=None)) # to check the floating point round off
# print('betas1_actual:',betas1_actual)

# betas2_pseudo = np.arange(0, 1, 0.2)
# betas2_pseudo_range = [0, 1.0]
# betas2_range = [0.5, 0.9999]
# betas2_actual = element_wise_scaler(betas2_pseudo,feature_range=betas2_range,data_range=betas2_pseudo_range)
# betas2_actual = np.float64(np.around(betas2_actual, decimals=4, out=None)) # to check the floating point round off
# print('betas2_actual:',betas2_actual)


# epochs_step_pseudo = [0.5, 0.001, 0.25, 0.63, 0.65]
# epochs_step_pseudo_range = [0, 1.0]
# epochs_step_range = [5, 20]
# epochs_step_actual = element_wise_scaler(epochs_step_pseudo, feature_range=epochs_step_range, data_range=epochs_step_pseudo_range)
# epochs_step_actual = np.int64(np.round(epochs_step_actual))
# print('epochs_step_actual:',epochs_step_actual)

In [ ]:
# n = None

In [ ]:
# x=2
# if x==1:
#   a = 4
#   b = None
# else:
#   a = None
#   b = 4

In [ ]:
# print(a)

None


In [ ]:
# import pandas as pd
# from crabnet.crabnet_ import CrabNet
# from matbench.bench import MatbenchBenchmark


# cb = CrabNet(N=N_actual[0], epochs_step=5, epochs=20, losscurve=False, learningcurve=False)
# mb = MatbenchBenchmark(autoload=False, subset = ["matbench_expt_gap"])


# for task in mb.tasks:
#     task.load()
#     for fold in task.folds:

#         # Inputs are either chemical compositions as strings
#         # or crystal structures as pymatgen.Structure objects.
#         # Outputs are either floats (regression tasks) or bools (classification tasks)
#         train_inputs, train_outputs = task.get_train_and_val_data(fold)

#         # prep input for CrabNet
#         train_df = pd.concat((train_inputs, train_outputs), axis=1, keys=["formula", "target"])

#         if dummy:
#           train_df = train_df.head(10)

#         # train and validate your model
#         cb.fit(train_df = train_df)

#         # Get testing data
#         test_inputs, test_outputs = task.get_test_data(fold, include_target=True)
#         test_df = pd.concat((test_inputs, test_outputs), axis=1, keys=["formula", "target"])

#         # Predict on the testing data
#         # Your output should be a pandas series, numpy array, or python iterable
#         # where the array elements are floats or bools
#         predictions = cb.predict(test_df = test_df)

#         # Record your data!
#         task.record(fold, predictions)

# # Save your results
# mb.to_file("results.json.gz")
